<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/LANGCHAIN_AAI_GEMINI_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_core langchain_google_genai langgraph -q

In [11]:
import os
import asyncio
from typing import TypedDict, Annotated, List
import operator
from langchain_core.messages import BaseMessage
from langgraph.graph import StateGraph, END
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# --- Google Gemini API Configuration ---
import google.generativeai as genai

class AgentConfig:
    LLM_MODEL_NAME: str = "gemini-1.5-flash"

GOOGLE_API_KEY = None
try:
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GEMINI')
    print("Google Generative AI configured successfully using Colab Secrets.")
except (ImportError, KeyError):
    print("Not running in Google Colab or 'GEMINI' secret not found. Attempting to get 'GEMINI' environment variable.")
    GOOGLE_API_KEY = os.getenv('GEMINI')

if GOOGLE_API_KEY:
    genai.configure(api_key=GOOGLE_API_KEY)
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print(f"Gemini API configured with model: {AgentConfig.LLM_MODEL_NAME}")
else:
    print("Warning: GOOGLE_API_KEY not found. LLM calls will not work.")
    print("Please set your 'GEMINI' environment variable or Colab secret.")

from langchain_google_genai import ChatGoogleGenerativeAI

# --- 1. Define the Agent State ---
class AgentState(TypedDict):
    """
    Represents the state of our flight planning agent.
    """
    user_input: str
    flight_search_query: str
    flight_results: List[str]
    booking_status: str
    next_action: str # Used by LLM to decide
    previous_action_failed: bool # Flag to indicate if last action failed
    flights_found_this_turn: bool # Flag to indicate if flights have already been found in this session

# --- 2. Initialize the LLM (Gemini 1.5 Flash) ---
llm = ChatGoogleGenerativeAI(model=AgentConfig.LLM_MODEL_NAME, temperature=0.7)

# --- CRITICAL PROMPT UPDATE ---
DECISION_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", """You are a flight planning assistant. Your goal is to help the user find and potentially book flights.
         Based on the user's current request and the accumulated state of our conversation, decide the single best next action.

         Available actions:
         - `parse_user_input`: Use this if the 'user_input' is a new, general request or needs detailed extraction for flight details.
         - `search_flights`: Use this ONLY if the user explicitly asks to find flights, AND a valid 'flight_search_query' (with origin AND destination) is present, AND 'flights_found_this_turn' is False.
         - `book_flight`: Use this if the user explicitly asks to book a flight, AND 'flight_results' are available, AND 'booking_status' is not 'confirmed' or 'canceled'.
         - `cancel_booking`: Use this if the user clearly wants to cancel a booking, AND 'booking_status' is 'confirmed'.
         - `clarify_request`: Use this if the 'user_input' is unclear, OR a previous `search_flights` or `book_flight` action failed, OR 'flight_search_query' is 'N/A' when a search is implied.
         - `end_conversation`: Use this if the user's request is fulfilled (e.g., flights found and presented, booking confirmed), the query is completely irrelevant to flight planning, or the user explicitly indicates they are done.

         Current State:
         User Input: {user_input}
         Flight Search Query: {flight_search_query}
         Flight Results: {flight_results}
         Booking Status: {booking_status}
         Previous Action Failed: {previous_action_failed}
         Flights Found This Turn: {flights_found_this_turn}

         Instructions:
         1. **HIGHEST PRIORITY: Task Completion**:
            - If 'booking_status' is 'confirmed', the booking task is definitively complete, so you MUST choose `end_conversation`.
            - If 'flights_found_this_turn' is True and the 'user_input' does NOT contain explicit "book" or "cancel" commands, choose `end_conversation`. This indicates information was provided, and you are awaiting the user's next explicit command.
         2. **PRIORITY 2: Error/Clarification**: If 'previous_action_failed' is True, OR if 'flight_search_query' is 'N/A' and a flight search or booking intent is present in 'user_input', you MUST choose `clarify_request`. Do NOT attempt `search_flights` or `book_flight` again without clarification.
         3. **PRIORITY 3: Irrelevant Input**: If 'user_input' is completely unrelated to flight planning, choose `end_conversation`.
         4. **Otherwise**: Carefully select the most appropriate action based on the 'user_input' and the rest of the 'Current State'.

         What is the ONE best action to take next? Respond with ONLY the action name, no other text.
         For example: `search_flights`
         """),
        ("human", "{user_input}"),
    ]
)

decision_chain = DECISION_PROMPT | llm | StrOutputParser()

# --- 3. Define the Agent's Nodes (Actions) ---

# --- IMPROVED parse_user_input ---
def parse_user_input(state: AgentState) -> AgentState:
    """
    Node to parse the raw user input into a structured flight search query.
    """
    print(f"--- PARSING USER INPUT: '{state.get('user_input', '')}' ---")
    user_input = state.get("user_input", "").lower()

    query = "N/A - No specific flight search query" # Default to N/A

    # Determine if user input is a command (book/cancel) or a search request
    is_book_command = "book" in user_input and ("flight" in user_input or "one" in user_input)
    is_cancel_command = "cancel" in user_input and ("booking" in user_input or "flight" in user_input)
    is_search_request = "flight" in user_input or "flights" in user_input or "travel" in user_input or "trip" in user_input or "find" in user_input or "show" in user_input or "search" in user_input or "look" in user_input

    if is_book_command or is_cancel_command:
        # For commands, we don't need to re-extract a flight_search_query;
        # the LLM should use existing state or decide based on the command intent.
        query = state.get("flight_search_query", "N/A - Command")
    elif is_search_request:
        has_from = "from" in user_input
        has_to = "to" in user_input

        if has_from and has_to:
            from_index = user_input.find("from")
            to_index = user_input.find("to", from_index)
            if from_index != -1 and to_index != -1 and to_index > from_index:
                origin_part = user_input[from_index + 4:to_index].strip()
                destination_part = user_input[to_index + 2:].strip().split(' ')[0].replace('.', '')
                query = f"flight from {origin_part} to {destination_part}"
            else:
                query = "N/A - Incomplete from/to search"
        elif has_to:
            destination_part = user_input.split("flight to", 1)[-1].strip().split(' ')[0].replace('.', '')
            query = f"N/A - Missing origin for flight to {destination_part}"
        else:
            query = "N/A - General search, missing details"

    else:
        query = "N/A - Not a flight-related request"


    print(f"Parsed flight query: {query}")
    return {"flight_search_query": query, "previous_action_failed": False, "flights_found_this_turn": False}


async def decide_next_action_llm(state: AgentState) -> AgentState:
    """
    Uses the LLM to decide the next action based on the current state.
    """
    print("--- LLM DECIDING NEXT ACTION ---")

    llm_input = {
        "user_input": state.get("user_input", ""),
        "flight_search_query": state.get("flight_search_query", ""),
        "flight_results": str(state.get("flight_results", [])),
        "booking_status": state.get("booking_status", "N/A"),
        "previous_action_failed": state.get("previous_action_failed", False),
        "flights_found_this_turn": state.get("flights_found_this_turn", False)
    }

    action = await decision_chain.ainvoke(llm_input)
    action = action.strip().replace("`", "")

    print(f"LLM decided action: {action}")
    return {"next_action": action}

# --- IMPROVED search_flights ---
def search_flights(state: AgentState) -> AgentState:
    """
    Simulates calling an external flight search API.
    """
    print("--- SEARCHING FLIGHTS ---")
    query = state.get("flight_search_query", "N/A")
    print(f"Searching for: {query}")

    if query and query != "N/A" and "from" in query and "to" in query:
        simulated_results = [
            f"Flight AA101 {query} on {os.environ.get('CURRENT_DATE', 'a future date')} at 9 AM",
            f"Flight BA202 {query} on {os.environ.get('CURRENT_DATE', 'a future date')} at 2 PM"
        ]
        print(f"Found flights: {simulated_results}")
        # Mark flights as found this turn
        return {"flight_results": simulated_results, "booking_status": "pending", "previous_action_failed": False, "flights_found_this_turn": True}
    else:
        print("Invalid or incomplete flight search query. Cannot search. Setting previous_action_failed.")
        return {"flight_results": [], "booking_status": "failed", "previous_action_failed": True, "next_action": "clarify_request", "flights_found_this_turn": False}


# --- IMPROVED book_flight ---
def book_flight(state: AgentState) -> AgentState:
    """
    Simulates calling an external flight booking API.
    """
    print("--- ATTEMPTING TO BOOK FLIGHT ---")
    results = state.get("flight_results", [])
    if results and state.get("booking_status") != "confirmed" and state.get("booking_status") != "canceled":
        selected_flight = results[0]
        print(f"Attempting to book: {selected_flight}")
        print("Flight successfully booked!")
        # CRITICAL: Set next_action to end_conversation on success
        return {"booking_status": "confirmed", "previous_action_failed": False, "next_action": "end_conversation"}
    else:
        print("Cannot book. No flights found, or booking already confirmed/canceled. Setting previous_action_failed.")
        return {"booking_status": "failed", "previous_action_failed": True, "next_action": "clarify_request"}

def cancel_booking(state: AgentState) -> AgentState:
    """
    Simulates canceling a flight booking.
    """
    print("--- CANCELING BOOKING ---")
    status = state.get("booking_status", "")
    if status == "confirmed":
        print("Booking canceled successfully.")
        return {"booking_status": "canceled", "previous_action_failed": False}
    else:
        print("No confirmed booking to cancel. Setting previous_action_failed.")
        return {"booking_status": "failed", "previous_action_failed": True}

def clarify_request(state: AgentState) -> AgentState:
    """
    Prompts the user for clarification.
    """
    print("--- CLARIFYING REQUEST ---")
    print(f"Current state user input: '{state.get('user_input', 'N/A')}'")
    print(f"Current parsed query: '{state.get('flight_search_query', 'N/A')}'")
    print(f"Previous action failed: {state.get('previous_action_failed', False)}")
    print("I need more information to help you. Could you please specify your origin, destination, and dates?")
    return {"next_action": "parse_user_input", "previous_action_failed": False, "flights_found_this_turn": False} # Reset flags

# --- 4. Define the Graph and Edges ---

workflow = StateGraph(AgentState)

workflow.add_node("parse_user_input", parse_user_input)
workflow.add_node("decide_next_action", decide_next_action_llm)
workflow.add_node("search_flights", search_flights)
workflow.add_node("book_flight", book_flight)
workflow.add_node("cancel_booking", cancel_booking)
workflow.add_node("clarify_request", clarify_request)

workflow.set_entry_point("parse_user_input")

workflow.add_edge("parse_user_input", "decide_next_action")

workflow.add_conditional_edges(
    "decide_next_action",
    lambda state: state["next_action"],
    {
        "search_flights": "search_flights",
        "book_flight": "book_flight",
        "cancel_booking": "cancel_booking",
        "clarify_request": "clarify_request",
        "end_conversation": END,
        "parse_user_input": "parse_user_input",
    },
)

# Route to decide_next_action after search_flights completes
workflow.add_edge("search_flights", "decide_next_action")

# --- CRITICAL CHANGE: book_flight now routes to decide_next_action so LLM can choose END ---
workflow.add_edge("book_flight", "decide_next_action")

workflow.add_edge("cancel_booking", END)

# --- CRITICAL CHANGE: clarifiy_request now goes to END for the turn ---
workflow.add_edge("clarify_request", END)


# --- 5. Compile the Graph ---
app = workflow.compile()

# --- 6. Helper function to run scenarios ---
async def run_scenario(scenario_name: str, inputs: List[dict]):
    """
    Runs a given scenario, handling multi-turn interactions and printing state.
    """
    print(f"\n{'='*10} Running the Agent: {scenario_name} {'='*10}")
    # Reset state for each new scenario run
    current_state = {"previous_action_failed": False, "flight_results": [], "booking_status": "N/A", "flight_search_query": "N/A", "flights_found_this_turn": False}

    for i, user_input_dict in enumerate(inputs):
        print(f"\n--- User Turn {i+1} ---")
        current_state.update(user_input_dict)
        print(f"Initial State for Turn {i+1}:\n{current_state}")
        try:
            final_turn_state = await app.ainvoke(current_state)
            print(f"Final State after Turn {i+1}:\n{final_turn_state}")
            current_state = final_turn_state
        except Exception as e:
            print(f"Error during turn {i+1}: {e}")
            break
        print("-" * 30)
    print("=" * (20 + len(scenario_name)))


# --- 7. Main execution logic ---
async def main():
    # Set the current date for simulated results
    # Current time is Thursday, July 3, 2025 at 12:56:49 AM EDT.
    os.environ['CURRENT_DATE'] = "July 3, 2025"

    # --- ONLY ONE TEST CASE FOR TESTING ---
    await run_scenario(
        "1.1: Basic Search & Book (Multi-turn)",
        [
            {"user_input": "Find me a flight from Montreal to Vancouver next Tuesday."},
            {"user_input": "Book the earliest one."},
        ]
    )


# --- Final execution for Colab/Jupyter ---
await main()

Google Generative AI configured successfully using Colab Secrets.
Gemini API configured with model: gemini-1.5-flash

========== Running the Agent: 1.1: Basic Search & Book (Multi-turn) ==========

--- User Turn 1 ---
Initial State for Turn 1:
{'previous_action_failed': False, 'flight_results': [], 'booking_status': 'N/A', 'flight_search_query': 'N/A', 'flights_found_this_turn': False, 'user_input': 'Find me a flight from Montreal to Vancouver next Tuesday.'}
--- PARSING USER INPUT: 'Find me a flight from Montreal to Vancouver next Tuesday.' ---
Parsed flight query: flight from montreal to vancouver
--- LLM DECIDING NEXT ACTION ---
LLM decided action: search_flights
--- SEARCHING FLIGHTS ---
Searching for: flight from montreal to vancouver
Found flights: ['Flight AA101 flight from montreal to vancouver on July 3, 2025 at 9 AM', 'Flight BA202 flight from montreal to vancouver on July 3, 2025 at 2 PM']
--- LLM DECIDING NEXT ACTION ---
LLM decided action: end_conversation
Final State afte